In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1646235494062_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = spark.read.load("/user/root/nord_bank/part-m-00000.parquet", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+------------------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+-------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency|         card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|   temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+------------------+------------------+----------+-

In [4]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [5]:
expr = """
    CASE
        WHEN month LIKE \'%January%\' THEN \'1\'
        WHEN month LIKE \'%February%\' THEN \'2\'
        WHEN month LIKE \'%March%\' THEN \'3\'
        WHEN month LIKE \'%April%\' THEN \'4\'
        WHEN month LIKE \'%May%\' THEN \'5\'
        WHEN month LIKE \'%June%\' THEN \'6\'
        WHEN month LIKE \'%July%\' THEN \'7\'
        WHEN month LIKE \'%August%\' THEN \'8\'
        WHEN month LIKE \'%September%\' THEN \'9\'
        WHEN month LIKE \'%October%\' THEN \'10\'
        WHEN month LIKE \'%November%\' THEN \'11\'
        WHEN month LIKE \'%December%\' THEN \'12\'
        ELSE \'other_unknown\'
    END AS Month_int"""

df = df.selectExpr("*", expr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number
df = df.withColumn("full_date_time",F.expr("make_timestamp(year, Month_int, day, hour, 0, 0)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**DIM_DATE**

In [7]:
DIM_DATE = df.select("full_date_time","year", "month", "day", "weekday", "hour")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
DIM_DATE = DIM_DATE.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
DIM_DATE.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [10]:
DIM_DATE = DIM_DATE.select("*", F.row_number().over(Window.partitionBy().orderBy(DIM_DATE['full_date_time'])).alias("date_id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
DIM_DATE = DIM_DATE.select("date_id","full_date_time","year", "month", "day", "hour", "weekday")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
DIM_DATE.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date_id: integer (nullable = true)
 |-- full_date_time: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- weekday: string (nullable = true)

In [13]:
DIM_DATE.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+-------+---+----+-------+
|date_id|     full_date_time|year|  month|day|hour|weekday|
+-------+-------------------+----+-------+---+----+-------+
|      1|2017-01-01 00:00:00|2017|January|  1|   0| Sunday|
|      2|2017-01-01 01:00:00|2017|January|  1|   1| Sunday|
|      3|2017-01-01 02:00:00|2017|January|  1|   2| Sunday|
|      4|2017-01-01 03:00:00|2017|January|  1|   3| Sunday|
|      5|2017-01-01 04:00:00|2017|January|  1|   4| Sunday|
|      6|2017-01-01 05:00:00|2017|January|  1|   5| Sunday|
|      7|2017-01-01 06:00:00|2017|January|  1|   6| Sunday|
|      8|2017-01-01 07:00:00|2017|January|  1|   7| Sunday|
|      9|2017-01-01 08:00:00|2017|January|  1|   8| Sunday|
|     10|2017-01-01 09:00:00|2017|January|  1|   9| Sunday|
|     11|2017-01-01 10:00:00|2017|January|  1|  10| Sunday|
|     12|2017-01-01 11:00:00|2017|January|  1|  11| Sunday|
|     13|2017-01-01 12:00:00|2017|January|  1|  12| Sunday|
|     14|2017-01-01 13:00:00|2017|Januar

In [14]:
DIM_DATE.write.option("header", "true").csv("s3://etlbukcetkshitiz/ETL_UPGRAD/DIM_DATE/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**DIM_LOCATION**

In [15]:
DIM_LOCATION = df.select("atm_location", "atm_streetname", "atm_street_number", "atm_zipcode", "atm_lat", "atm_lon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
DIM_LOCATION = DIM_LOCATION.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
DIM_LOCATION.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [18]:
DIM_LOCATION = DIM_LOCATION.select("*", F.row_number().over(Window.partitionBy().orderBy(DIM_LOCATION['atm_location'])).alias("location_id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
DIM_LOCATION = DIM_LOCATION.withColumn("atm_lon", DIM_LOCATION["atm_lon"].cast(DecimalType(10,3)))
DIM_LOCATION = DIM_LOCATION.withColumn("atm_lat", DIM_LOCATION["atm_lat"].cast(DecimalType(10,3)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
DIM_LOCATION = DIM_LOCATION.withColumnRenamed("atm_location", "location")\
.withColumnRenamed("atm_streetname", "street_name")\
.withColumnRenamed("atm_street_number", "street_number")\
.withColumnRenamed("atm_zipcode", "zipcode")\
.withColumnRenamed("atm_lat", "lat")\
.withColumnRenamed("atm_lon", "lon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
DIM_LOCATION = DIM_LOCATION.select("location_id","location","street_name", "street_number", "zipcode", "lat", "lon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
DIM_LOCATION.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- location_id: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- street_number: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: decimal(10,3) (nullable = true)
 |-- lon: decimal(10,3) (nullable = true)

In [23]:
DIM_LOCATION.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------------+-------------+-------+------+------+
|location_id|            location|       street_name|street_number|zipcode|   lat|   lon|
+-----------+--------------------+------------------+-------------+-------+------+------+
|          1|             Aabybro|      ÃƒËœstergade|            6|   9440|57.162| 9.730|
|          2|      Aalborg Hallen|      Europa Plads|            4|   9000|57.044| 9.913|
|          3|Aalborg Storcente...|          Hobrovej|          452|   9200|57.005| 9.876|
|          4|Aalborg Storcente...|          Hobrovej|          452|   9200|57.005| 9.876|
|          5|         Aalborg Syd|          Hobrovej|          440|   9200|57.005| 9.881|
|          6|           AalbÃƒÂ¦k|        Centralvej|            5|   9982|57.593|10.412|
|          7|              Aarhus|        Ceres Byen|           75|   8000|56.157|10.194|
|          8|              Aarhus|    SÃƒÂ¸nder Alle|           11|   8000|56.153|10.206|
|         

In [24]:
DIM_LOCATION.write.option("header", "true").csv("s3://etlbukcetkshitiz/ETL_UPGRAD/DIM_LOCATION/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**DIM_ATM**

In [25]:
DIM_ATM = df.select("atm_id", "atm_manufacturer", "atm_location", "atm_streetname")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
DIM_ATM = DIM_ATM.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
DIM_ATM.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [28]:
DIM_ATM = DIM_ATM.withColumn("atm_number", DIM_ATM["atm_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
DIM_ATM = DIM_ATM.join(DIM_LOCATION, 
               (DIM_ATM.atm_location == DIM_LOCATION.location) &  (DIM_ATM.atm_streetname == DIM_LOCATION.street_name),
               "leftouter")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
DIM_ATM = DIM_ATM.withColumnRenamed("location_id", "atm_location_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
DIM_ATM = DIM_ATM.select("atm_id", "atm_number", "atm_manufacturer", "atm_location_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
DIM_ATM.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- atm_id: string (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location_id: integer (nullable = true)

In [33]:
DIM_ATM.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+---------------+
|atm_id|atm_number|atm_manufacturer|atm_location_id|
+------+----------+----------------+---------------+
|    82|        82|             NCR|             83|
|    45|        45|             NCR|             11|
|    86|        86|             NCR|             36|
|   110|       110| Diebold Nixdorf|             41|
|   108|       108|             NCR|             47|
|    97|        97|             NCR|             51|
|   101|       101|             NCR|             17|
|    30|        30|             NCR|             72|
|     6|         6|             NCR|             25|
|    91|        91|             NCR|             87|
|    40|        40| Diebold Nixdorf|             27|
|    90|        90|             NCR|             59|
|    76|        76|             NCR|             19|
|   102|       102|             NCR|              3|
|    61|        61|             NCR|             23|
|    55|        55|             NCR|          

In [34]:
DIM_ATM.write.option("header", "true").csv("s3://etlbukcetkshitiz/ETL_UPGRAD/DIM_ATM/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**DIM_CARD_TYPE**

In [35]:
DIM_CARD_TYPE = df.select("card_type")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
DIM_CARD_TYPE = DIM_CARD_TYPE.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
DIM_CARD_TYPE.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [38]:
DIM_CARD_TYPE = DIM_CARD_TYPE.select("*", F.row_number().over(Window.partitionBy().orderBy(DIM_CARD_TYPE['card_type'])).alias("card_type_id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
DIM_CARD_TYPE = DIM_CARD_TYPE.select("card_type_id", "card_type")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
DIM_CARD_TYPE.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- card_type_id: integer (nullable = true)
 |-- card_type: string (nullable = true)

In [41]:
DIM_CARD_TYPE.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|card_type_id|           card_type|
+------------+--------------------+
|           1|              CIRRUS|
|           2|             Dankort|
|           3|     Dankort - on-us|
|           4|         HÃƒÂ¦vekort|
|           5| HÃƒÂ¦vekort - on-us|
|           6|             Maestro|
|           7|          MasterCard|
|           8|  Mastercard - on-us|
|           9|                VISA|
|          10|        Visa Dankort|
|          11|Visa Dankort - on-us|
|          12|            VisaPlus|
+------------+--------------------+

In [42]:
DIM_CARD_TYPE.write.option("header", "true").csv("s3://etlbukcetkshitiz/ETL_UPGRAD/DIM_CARD_TYPE/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**FACT_ATM_TRANS**

In [43]:
FACT_ATM_TRANS = df.select("atm_id","atm_location","atm_streetname","full_date_time","card_type","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [45]:
FACT_ATM_TRANS=FACT_ATM_TRANS.select("*", F.row_number().over(Window.partitionBy().orderBy(FACT_ATM_TRANS['transaction_amount'])).alias("trans_id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
FACT_ATM_TRANS.select("trans_id").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [47]:
FACT_ATM_TRANS = FACT_ATM_TRANS.join(DIM_DATE, 
               FACT_ATM_TRANS.full_date_time == DIM_DATE.full_date_time, 
               "leftouter")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [49]:
FACT_ATM_TRANS = FACT_ATM_TRANS.select("trans_id","atm_id","atm_location","atm_streetname","date_id","card_type","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
FACT_ATM_TRANS = FACT_ATM_TRANS.join(DIM_LOCATION, 
               (FACT_ATM_TRANS.atm_location == DIM_LOCATION.location) 
                                     & 
                (FACT_ATM_TRANS.atm_streetname == DIM_LOCATION.street_name),
               "leftouter")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [52]:
FACT_ATM_TRANS = FACT_ATM_TRANS.withColumnRenamed("location_id", "weather_loc_id")
FACT_ATM_TRANS = FACT_ATM_TRANS.select("trans_id","atm_id","weather_loc_id","date_id","card_type","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [54]:
FACT_ATM_TRANS = FACT_ATM_TRANS.join(DIM_CARD_TYPE, 
               FACT_ATM_TRANS.card_type == DIM_CARD_TYPE.card_type, 
               "leftouter")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [56]:
FACT_ATM_TRANS = FACT_ATM_TRANS.select("trans_id","atm_id","weather_loc_id","date_id","card_type_id","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id","weather_main","weather_description")
FACT_ATM_TRANS.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+--------------------+-------+----------+----------+------------+--------------------+
|trans_id|atm_id|weather_loc_id|date_id|card_type_id|atm_status|currency|   service|transaction_amount|message_code|        message_text|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+--------------------+-------+----------+----------+------------+--------------------+
|
|       2|    78|            71|    272|          11|    Active|     DKK|Withdrawal|                 0|            |                    |  0.000|        20|       521|        Rain|proximity shower ...|
|
|
|
|       6|    19|            62|    438|          11|    Active|     DKK|Withdrawal|                 0|            |                    |  0.000|        90|       300|     Drizzle|ligh

In [57]:
FACT_ATM_TRANS = FACT_ATM_TRANS.withColumn("rain_3h", FACT_ATM_TRANS["rain_3h"].cast(DecimalType(10,3)))
FACT_ATM_TRANS = FACT_ATM_TRANS.withColumn("trans_id", FACT_ATM_TRANS["trans_id"].cast(LongType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
FACT_ATM_TRANS.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trans_id: long (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- weather_loc_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- card_type_id: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- service: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- rain_3h: decimal(10,3) (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)

In [59]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [60]:
FACT_ATM_TRANS.write.option("header", "true").csv("s3://etlbukcetkshitiz/ETL_UPGRAD/FACT_ATM_TRANS/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…